In [ ]:
import pandas as pd

In [ ]:
import hyperopt as hp

In [ ]:
import catboost as cb
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('datasets/train_sample.csv', encoding='utf8', index_col=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(strat_train_set[cat_cols], strat_train_set.click, test_size=0.33, random_state=27)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
cat_cols = ['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C15', 'C16', 'C18', 'C20', 'C21']

In [ ]:
dataset = cb.Pool(X_train, y_train, cat_features=cat_cols)

In [ ]:
test_dataset = cb.Pool(X_test, y_test, cat_features=cat_cols)

In [ ]:
cv_dataset = cb.Pool(data=X_train.head(100),
                     label=y_train.head(100),
                     cat_features=cat_cols)

In [ ]:
cv_params = {
    "loss_function": "CrossEntropy",
    "max_depth": 3,
    "n_estimators": 10,  #[0,1,2,3,4,5] -> [50,]
    "learning_rate": 0.01,  #[0,1,2,3,4,5] -> 0.05,0.06
}

In [ ]:
def CatOptCV(params):
    model = cb.cv(pool=cv_dataset,
                  params=params,
                  early_stopping_rounds=10,
                  fold_count=5,
                  plot="True")
    

    return model

In [ ]:
def CatOpt(params):
    model = cb.CatBoostClassifier(loss_function='CrossEntropy',
#                                   custom_metric='F1',
#                                   eval_metric='F1',
#                                   l2_leaf_reg=3,
                                  early_stopping_rounds=10,
                                  task_type="CPU",
                                  **params)
    model.fit(X_train,
              y_train,
              cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8],
              plot="True")
    
    preds_class = model.predict(X_test)
    return f1_score(y_test, preds_class, zero_division=1)

In [ ]:
%%time
params = {
    "max_depth": hp.hp.randint("max_depth", 15),
    "n_estimators": hp.hp.randint("n_estimators", 1000),
    "learning_rate": hp.hp.loguniform("learning_rate", np.log(0.01), np.log(0.2)),
}
trials = hp.Trials()
# algo = hp.partial(hp.tpe.suggest, n_startup_jobs=1)
best = hp.fmin(CatOpt,
               space=params,
               algo=hp.tpe.suggest,
               trials=trials,
               max_evals=5)
print('best:')
best

In [ ]:
def trainning(params):
    model = cb.CatBoostClassifier(loss_function='CrossEntropy',
                                  custom_metric='F1',
                                  eval_metric='F1',
                                  l2_leaf_reg=3,
                                  early_stopping_rounds=10,
                                  task_type="CPU",
                                  **params)
    model.fit(X_train,
              y_train,
              cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 8],
              plot="True")
    return model